In [1]:
import os 
import os.path as op
import math
import numpy as np
import mne
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
%matplotlib qt

In [2]:
subjects = ['n1', 'n2', 'n3', 'n4', 'n5', 'n10']
path = '/home/jerry/python_projects/space/closedloop/test_data'
raw_fname = op.join(path, '{0}_raw.fif')
eve_fname = op.join(path, '{0}_eve.fif')

Gtk-Message: 21:49:00.677: Failed to load module "canberra-gtk-module"
Gtk-Message: 21:49:00.677: Failed to load module "canberra-gtk-module"


In [3]:
raw = mne.io.read_raw_fif(raw_fname.format(subjects[0]))
print(type(raw))
print(raw.ch_names[2])
print(type(None))


Opening raw data file /home/jerry/python_projects/space/closedloop/test_data/n1_raw.fif...
    Range : 0 ... 17309999 =      0.000 ... 34619.998 secs
Ready.
<class 'mne.io.fiff.raw.Raw'>
F2-F4
<class 'NoneType'>


In [4]:
raw.ch_names

['ROC-LOC',
 'LOC-ROC',
 'F2-F4',
 'F4-C4',
 'C4-P4',
 'P4-O2',
 'F1-F3',
 'F3-C3',
 'C3-P3',
 'P3-O1',
 'C4-A1',
 'EMG1-EMG2',
 'ECG1-ECG2',
 'staging']

In [5]:
events = mne.read_events(eve_fname.format(subjects[0]))
print(events)

[[  270000        0        1]
 [  330000        0        2]
 [  375000        0        3]
 [  615000        0        2]
 [  645000        0        3]
 [ 2130000        0        2]
 [ 2370000        0        1]
 [ 2385000        0        2]
 [ 2700000        0        4]
 [ 3030000        0        2]
 [ 3360000        0        3]
 [ 3750000        0        0]
 [ 3840000        0        1]
 [ 3855000        0        2]
 [ 4320000        0        3]
 [ 5685000        0        2]
 [ 5745000        0        0]
 [ 5790000        0        1]
 [ 5835000        0        2]
 [ 6015000        0        4]
 [ 6765000        0        2]
 [ 7522500        0       -1]
 [ 7530000        0        1]
 [ 7560000        0        2]
 [ 7844000        0       -1]
 [ 7845000        0        1]
 [ 7860000        0        2]
 [ 8040000        0        3]
 [ 8385000        0        2]
 [ 8445000        0        3]
 [ 8925000        0        0]
 [ 8955000        0        1]
 [ 8985000        0        2]
 [ 9060000

In [11]:
def data_streamer(raw_data, events, channels=None, stages=[2, 3], buffer_len=50.):
    # Check function arguments are correct:
    assert isinstance(raw_data, mne.io.fiff.raw.Raw), AssertionError('raw shoul \
                                                                be a mne.Raw \
                                                                object')
    
    if channels is None:
        channels = raw_data.ch_names
    elif isinstance(channels, list):
        if all(isinstance(c, int) for c in channels):
            channels = [raw_data.ch_names[c] for c in channels]
        elif all(isinstance(c, str) for c in channels):
            assert all(c in raw_data.ch_names for c in channels), AssertionError
        else:
            raise ValueError('Channels shoul be list of strings, \
                             list of int or None')
    else:
        raise ValueError('Channels shoul be list of strings, \
                             list of int or None')
    
    if isinstance(events, np.ndarray):
        if not (events.ndim == 2 and events.shape[1] == 3):
            raise ValueError('events should be a [n, 3] dimensional array')
    else:
        raise ValueError('events should be a [n, 3] dimensional array')

    # Creating staging vector
    staging = raw_data.pick(['staging'])
    staging = staging.get_data()
    ev_start = 0
    for i, e in enumerate(events):
        if i == 0:
            staging[ev_start:e[0]] = staging[0]
            ev_start = e[0]
        elif i == len(events[0, :]):
            staging[ev_start:len(staging)] = events[-1, -1]
        else:
            staging[ev_start:e[0]] = events[i - 1, -1]
            ev_start = e[0]
            
    # Selecting desired channels (get rid of staging)
    raw_data = raw_data.pick_channels(channels, ordered=True, verbose=False)
    print('Keeping channels:', raw_data.ch_names)

    # Computing buffer sample length 
    sfreq = raw_data.info['sfreq']
    n_sample = int((sfreq / 1000) * buffer_len) # samples each ms * buffer ms
    print('Number of samples in a buffer:', n_sample)

    raw_data = xr.DataArray(raw_data.get_data(), 
                            coords=[raw_data.ch_names, staging], 
                            dims=['channel', 'stage'])
    
    s_start = int(0)
    s_end = n_sample
    s_steps = math.ceil(raw_data.shape[-1] / n_sample)
    for s in range(s_steps):
        chunk = raw_data[:, s_start:s_end]
        if chunk.stage[0] not in stages:
            xr.full_like(chunk, np.nan)

        s_start += n_sample
        s_end += n_sample

        yield chunk
    # # Loop over time 
    #     # Check if looping in the desired stage
    #     # Chunk the data
    #     # yield the data (generator)
    # raw_tmax = raw.times[-1]
    # tmin = 0.
    # tmax = buffer_len / 1000
    # tsteps = math.ceil(raw_tmax / tmax)
    # tmin_sample = 0.
    # for t in tsteps:





In [12]:
chunk = data_streamer(raw, events, channels=['C4-A1'], stages=[2, 3], buffer_len=50.)

In [13]:
collect = []
collect.append(chunk)

In [14]:
collect

[<generator object data_streamer at 0x7f2e7cad31b0>]

In [15]:
next(chunk)

TypeError: 'method' object is not subscriptable